In [23]:
from openai import OpenAI
import numpy as np

client = OpenAI(api_key="sk-svcacct-oD2ywALJLsVNQwU3GZ-Lvl-6d53TV1hwjr7DKEfvT07SSYDzFHvmTo3Fx9gp-5KT3BlbkFJsc-MOn7QvehX14nXoONvR0tKAQ8_XBfDFywvS3CVLyL4QaCm9Pklco6v0i3hhAA")

#===== Creating an embedding with the openai api =======
# https://platform.openai.com/docs/guides/embeddings
response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-3-small"
)

In [24]:
query = "Who is Messi?"


resource1 = "Born in Rosario, Argentina, Messi relocated to Spain to join Barcelona at age 13, and made his competitive debut at age 17 in October 2004. He established himself as an integral player for the club within the next three years, and in his first uninterrupted season in 2008–09 helped Barcelona achieve the first treble in Spanish football; that year, aged 22, Messi won the first of his four consecutive Ballons d'Or, the first player to win it four times. During the 2011–12 season, he set La Liga and European records for most goals in a season, while establishing himself as Barcelona's all-time top scorer. The following two seasons, he finished second for the Ballon d'Or behind Cristiano Ronaldo, his perceived career rival, before regaining his best form during the 2014–15 campaign, where he became the all-time top scorer in La Liga, led Barcelona to a historic second treble, and won a fifth Ballon d'Or in 2015. Messi assumed captaincy of Barcelona in 2018, and won a record sixth Ballon d'Or in 2019. During his overall tenure at Barcelona, Messi won a club-record 34 trophies, including ten La Liga titles and four UEFA Champions Leagues, among others. He signed for French club Paris Saint-Germain in August 2021, where he would win the Ligue 1 title during both of his seasons there. Messi joined American club Inter Miami in July 2023, and set a new mark for most goals scored for the club by his second season."
resource2 = "An Argentine international, Messi is the national team's all-time leading goalscorer and most-capped player. His style of play as a diminutive, left-footed dribbler drew career-long comparisons with compatriot Diego Maradona, who described Messi as his successor. At the youth level, he won the 2005 FIFA World Youth Championship and gold medal at the 2008 Summer Olympics. After his senior debut in 2005, Messi became the youngest Argentine to play and score in a World Cup in 2006. He assumed the national team's captaincy in 2011, and then led Argentina to three consecutive finals: the 2014 FIFA World Cup, the 2015 Copa América and the Copa América Centenario, all of which they would lose. After initially announcing his international retirement in 2016, he returned to help his country narrowly qualify for the 2018 FIFA World Cup, which they would again exit early. Messi and the national team finally broke Argentina's 28-year trophy drought with a victory in the 2021 Copa América, where he was named the tournament's best player and which later helped him win his seventh Ballon d'Or that year. He then led Argentina to win the 2022 FIFA World Cup, his country's third overall world championship and first in 36 years. This followed with a record-extending eighth Ballon d'Or in 2023. A second Copa América victory with Messi as captain came in 2024."
resource3 = "Messi has endorsed sportswear company Adidas since 2006. According to France Football, he was the world's highest-paid footballer for five years out of six between 2009 and 2014, and was ranked the world's highest-paid athlete by Forbes in 2019 and 2022. Messi was among Time's 100 most influential people in the world in 2011, 2012, and 2023. In 2020 and 2023, he was named the Laureus World Sportsman of the Year, the first team-sport athlete to win it. In 2020, Messi was named to the Ballon d'Or Dream Team and became the second footballer and second team-sport athlete to surpass $1 billion in career earnings."

In [25]:
query_embedding = client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    ).data[0].embedding

resource_responses = client.embeddings.create(
            input=[resource1, resource2, resource3],
            model="text-embedding-3-small"
    ).data

resource1_embedding = resource_responses[0].embedding
resource2_embedding = resource_responses[1].embedding
resource3_embedding = resource_responses[2].embedding

query = np.array(query_embedding)

resource1_embedding = np.array(resource1_embedding)
resource2_embedding = np.array(resource2_embedding)
resource3_embedding = np.array(resource3_embedding)

def cosine_simularity(A, B):
    return np.dot(A,B) / ( np.linalg.norm(A) * np.linalg.norm(B) )

simularity1 = cosine_simularity(query_embedding, resource1_embedding)
simularity2 = cosine_simularity(query_embedding, resource2_embedding)
simularity3 = cosine_simularity(query_embedding, resource3_embedding)

print(f"Simularity of query and resource1: {simularity1}")
print(f"Simularity of query and resource2: {simularity2}")
print(f"Simularity of query and resource3: {simularity3}")

#simularity 1 has the highest value, so plug that resource into the large language model
new_prompt = "Who is Messi? Some resource: Born in Rosario, Argentina, Messi relocated to Spain to join Barcelona at age 13, and made his competitive debut at age 17 in October 2004. He established himself as an integral player for the club within the next three years, and in his first uninterrupted season in 2008–09 helped Barcelona achieve the first treble in Spanish football; that year, aged 22, Messi won the first of his four consecutive Ballons d'Or, the first player to win it four times. During the 2011–12 season, he set La Liga and European records for most goals in a season, while establishing himself as Barcelona's all-time top scorer. The following two seasons, he finished second for the Ballon d'Or behind Cristiano Ronaldo, his perceived career rival, before regaining his best form during the 2014–15 campaign, where he became the all-time top scorer in La Liga, led Barcelona to a historic second treble, and won a fifth Ballon d'Or in 2015. Messi assumed captaincy of Barcelona in 2018, and won a record sixth Ballon d'Or in 2019. During his overall tenure at Barcelona, Messi won a club-record 34 trophies, including ten La Liga titles and four UEFA Champions Leagues, among others. He signed for French club Paris Saint-Germain in August 2021, where he would win the Ligue 1 title during both of his seasons there. Messi joined American club Inter Miami in July 2023, and set a new mark for most goals scored for the club by his second season."
response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": new_prompt}])
print(response)

Simularity of query and resource1: 0.6529621525006128
Simularity of query and resource2: 0.6463148293312124
Simularity of query and resource3: 0.6117514740022656
ChatCompletion(id='chatcmpl-ATymKZeAkuFu7l8hILlqBxntcK05m', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Lionel Messi is a professional footballer widely regarded as one of the greatest players of all time. He is known for his incredible dribbling skills, vision, and goal-scoring ability. Messi has won numerous individual awards throughout his career, including multiple Ballon d'Or titles. He has also enjoyed great success with Barcelona, winning numerous domestic and international titles with the club. In 2021, Messi joined Paris Saint-Germain before moving to Inter Miami in 2023.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731708508, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerp